In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

RAGAS_APP_TOKEN = os.getenv("RAGAS_APP_TOKEN")
RAGAS_APP_TOKEN

'apt.4bbf-5ddb4071400c-59f6-a224-d773b65e-dd46d'

In [2]:
from langchain_community.document_loaders import FileSystemBlobLoader
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import PyMuPDFParser

loader = GenericLoader(
    blob_loader=FileSystemBlobLoader(
        path = "/Users/idotadmor/Downloads/Ono data",
        glob="*.pdf",
    ),
    blob_parser=PyMuPDFParser(),
)
docs = loader.load()

print(docs)

[Document(metadata={'producer': 'LibreOffice 25.2.0.3 (AARCH64) / LibreOffice Community', 'creator': 'Writer', 'creationdate': '2025-02-22T13:37:51+02:00', 'source': '/Users/idotadmor/Downloads/Ono data/שאלות ותשובות_.pdf', 'file_path': '/Users/idotadmor/Downloads/Ono data/שאלות ותשובות_.pdf', 'total_pages': 3, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': "D:20250222133751+02'00'", 'page': 0}, page_content=':שאלות ותשובות\nהרשמה ותנאי קבלה\n1\n.\n?איך אפשר להירשם למועדון היזמות\n○יש למלא טופס הרשמה (המתפרסם במידעון ובערוצי התקשורת של המרכז), לעבור ראיון \nאישי ולהתחייב להשתתפות פעילה.\n2\n.\n?האם יש תנאי קבלה מסוימים לסדנאות\n○חלק מהסדנאות פתוחות לכל הסטודנטים, אך לסדנאות מסוימות נדרש מילוי טופס הרשמה \nולעיתים גם ראיון קבלה.\n3\n.\n?האם אפשר להצטרף למפגשים בודדים או שחייבים להיות בכל המפגשים\n○\n ברוב הסדנאות יש\nנוכחות חובה\n בכל המפגשים כדי להפיק את המרב מהתהליך.\n4\n.\n?האם הסדנאות פתוחות

In [3]:
len(docs)

41

In [4]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))

/Users/idotadmor/V8/ragbasics/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from ragas.testset.transforms.extractors.llm_based import NERExtractor
from ragas.testset.transforms.splitters import HeadlineSplitter

transforms = [HeadlineSplitter(), NERExtractor()]

In [6]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)


In [7]:
from ragas.testset.synthesizers.single_hop.specific import (
    SingleHopSpecificQuerySynthesizer,
)

distribution = [
    (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 1.0),
]

for query, _ in distribution:
    prompts = await query.adapt_prompts("hebrew", llm=generator_llm)
    query.set_prompts(**prompts)

In [8]:
dataset = generator.generate_with_langchain_docs(
    docs[:],
    testset_size=10,
    transforms=transforms,
    query_distribution=distribution,
)

Applying HeadlineSplitter:   0%|          | 0/41 [00:00<?, ?it/s]unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transfo

ValueError: No nodes that satisfied the given filer. Try changing the filter.

In [ ]:
dataset.to_pandas()
dataset.upload()

In [8]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()

In [9]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )

In [10]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )

In [ ]:
kg.save("knowledge_graph.json")
loaded_kg = KnowledgeGraph.load("knowledge_graph.json")
loaded_kg

In [12]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings, knowledge_graph=loaded_kg)

In [13]:
from ragas.testset.synthesizers import default_query_distribution

query_distribution = default_query_distribution(generator_llm)

In [ ]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()